# Setup

In [ ]:
# Set Up Environment
import pandas as pd
import numpy as np
import seaborn as sns
import time
from scipy.ndimage import gaussian_filter
from kaggle_environments import evaluate, make, FailedPrecondition
from kaggle_environments.envs.halite.helpers import Configuration, Board, Observation
from src.coordinates import PointAlt
from src.orders import ShipOrder, BuildShipyardOrder, HarvestOrder
from src.gamestate import GameState
%matplotlib inline
env = make("halite", configuration={ "episodeSteps": 500 }, debug=True)
config = Configuration(env.configuration)
print (env.configuration)
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 8]
# automatically reimport when file has changed NOT working 
%load_ext autoreload
%autoreload 2

# Import agent

In [ ]:
from submission import agent
import submission

# Step by step debug

In [ ]:
# reset game
submission.commander = None
trainer = env.train([None, "random"])
observation = trainer.reset()
my_action = agent(observation, env.configuration)
print("action :", my_action)
print("Plans per id: ", GameState.planner.plan_x_id)
print("Planning", GameState.planner.planning)

In [ ]:
print(f"turn {GameState.board.step}")
print("----orders: ", submission.commander.orders)
print("----action :", my_action)
print("----Plans per id: ", GameState.planner.plan_x_id)
print("----Planning", GameState.planner.planning)
print("----positons:")
for s in GameState.board.current_player.ships:
    print(s.id, s.position.norm)

In [ ]:
# step
start_time = time.time()
num_steps = 1
try:
    for i in range(num_steps):
        observation, reward, done, info = trainer.step(my_action)
        my_action = agent(observation, env.configuration)
except FailedPrecondition:
    pass
    
env.render(mode="ipython", header=True, controls=True, width=650, height=550)
board = GameState.board
print("orders: ", submission.commander.orders)
print("action :", my_action)
print("positons:")
for s in board.current_player.ships:
    print(s.id, s.position.norm)
#print("harvesters_x_base: ", submission.commander.harvesters_x_base)
print(f"Execution time: {(time.time() - start_time) / num_steps} seconds")

In [ ]:
board.current_player.ship_ids

In [ ]:
#sns.heatmap(submission.commander.threat_map, cmap='Reds_r', linewidths=0.5, annot=False)
sns.heatmap(submission.commander.expansion_map, cmap='Blues', linewidths=0.5, annot=False)
#sns.heatmap(submission.commander.objectives_map, cmap='Reds_r', linewidths=0.5, annot=False)
#sns.heatmap(submission.commander.reward_map, cmap='RdYlGn', linewidths=0.5, annot=False)
#pd.DataFrame(submission.commander.threat_map).style.background_gradient(cmap='Reds_r').format('{:03,.0f}')

# Manual input

In [ ]:
my_action = {
    '0-1': 'WEST',
}
observation, reward, done, info = trainer.step(my_action)
env.render(mode="ipython", header=True, controls=True, width=650, height=550)

In [ ]:
ship = GameState.board.current_player.ships[0]
target = PointAlt(10, 19)

In [ ]:
submission.commander.orders[ship.id] = BuildShipyardOrder(ship, target)

# Run a game

In [ ]:
env.run([agent, "submission.py"])
env.render(mode="ipython", width=800, height=600)